# Function Calling

In [19]:
from openai import OpenAI
import json
from dotenv import dotenv_values

client:OpenAI = OpenAI(api_key=dotenv_values(".env").get("OPENAI_API_KEY"))
GPT_MODEL = "gpt-3.5-turbo-1106"

In [20]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def get_current_weather(location:str, unit:str = "fahrenheit")->str:
    """Get the current weather in a given location"""
    if "tokyo" == location.lower():
        return json.dumps({"location":"Tokyo", "temperature":"50", "unit":"fahrenheit"})
    elif "san francisco" == location.lower():
        return json.dumps({"location":"san francisco","temperature":"40", "unit":"fahrenheit" })
    else:
        return json.dumps({"location":"Pakistan","temperature":"40", "unit":"fahrenheit" })


In [95]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
from typing import Union

def run_conversation(main_request:str)->str:
    messages:list[dict[str, str]] = [{"role":"user", "content":main_request}]
    tools:list[dict[str, Union[str, dict]]] = [
        {
            "type":"function",
            "function":{
                "name":"get_current_weather",
                "description":"Get the current weather in a given location",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "location":{
                            "type":"string",
                            "description":"The city and state, e.g. San Francisco, CA"
                        },
                        "unit":{"type":"string","enum":["celsius","fahrenheit"]}
                    },
                    "required":["location"]
                }
            }
        }
    ]

    response:ChatCompletion = client.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    response_message:ChatCompletionMessage = response.choices[0].message
    display("* First Response: ",dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ",tool_calls)

    if tool_calls:
         available_functions={
              "get_current_weather":get_current_weather
         }

         messages.append(response_message)

         for tool_call in tool_calls:
             function_name = tool_call.function.name
             function_to_call = available_functions[function_name]
             function_args =json.loads(tool_call.function.arguments)
             function_response = function_to_call(
                 location = function_args.get("location"),
                 unit =  function_args.get("unit")
             )
             messages.append({
                 "tool_call_id":tool_call.id,
                 "role": "tool",
                 "name":function_name,
                 "content":function_response
             })
       
         second_response:ChatCompletion = client.chat.completions.create(
             model=GPT_MODEL,
             messages=messages,
         )

         display("* Second Response: ", dict(second_response))
         return second_response.choices[0].message.content



             
              




'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_LVxE59SRQKkSWdrzNfnla8M0', function=Function(arguments='{"location":"Pakistan","unit":"celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_LVxE59SRQKkSWdrzNfnla8M0', function=Function(arguments='{"location":"Pakistan","unit":"celsius"}', name='get_current_weather'), type='function')]

'* Second Response: '

{'id': 'chatcmpl-8SXJz7INKFhjr2TNE6WqKtaRGcxqO',
 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in Pakistan is 40 degrees Celsius (104 degrees Fahrenheit).', role='assistant', function_call=None, tool_calls=None))],
 'created': 1701811963,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_eeff13170a',
 'usage': CompletionUsage(completion_tokens=15, prompt_tokens=63, total_tokens=78)}

'The current weather in Pakistan is 40 degrees Celsius (104 degrees Fahrenheit).'

In [98]:
run_conversation("What's the weather like in Pakistan?")


'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_AjHiCV1tKhAWtJ72GCCougC3', function=Function(arguments='{"location":"Pakistan","unit":"celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_AjHiCV1tKhAWtJ72GCCougC3', function=Function(arguments='{"location":"Pakistan","unit":"celsius"}', name='get_current_weather'), type='function')]

'* Second Response: '

{'id': 'chatcmpl-8SXNdwn35VkISjF1CzE8XJKImnG29',
 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current temperature in Pakistan is 40 degrees Celsius (104 degrees Fahrenheit).', role='assistant', function_call=None, tool_calls=None))],
 'created': 1701812189,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_eeff13170a',
 'usage': CompletionUsage(completion_tokens=15, prompt_tokens=63, total_tokens=78)}

'The current temperature in Pakistan is 40 degrees Celsius (104 degrees Fahrenheit).'